In [1]:
from pymongo import MongoClient
import json
import glob
import os
import re
import datetime

In [2]:
# Function to import data from a JSON file
def import_weather_data(file_path, schema, collection, city):
    with open(file_path) as file:
        data = json.load(file)
        for item in data:
            item['city'] = city
            collection.insert_one(item)

In [3]:
# Function to import data from a JSON file
def import_futures_data(file_path, schema, collection, label):
    with open(file_path) as file:
        data = json.load(file)
        futures_data = []
        current_year = None
        for item in data:
            date_str = item['Date']
            year = datetime.datetime.strptime(date_str, "%Y-%m-%d").year
            if current_year is None:
                current_year = year
            elif year != current_year:
                # Process the futures data for the previous year
                process_futures_data(futures_data, schema, collection, label)
                futures_data = []
                current_year = year
            futures_data.append(item)
        
        # Process the last futures data
        process_futures_data(futures_data, schema, collection, label)

def process_futures_data(futures_data, schema, collection, label):
    # Insert the labeled data into the collection
    for item in futures_data:
        item['Label'] = label
    collection.insert_many(futures_data)

In [4]:
# Establish a connection to your MongoDB server
client = MongoClient('mongodb://localhost:27017/')
db = client['project_3_db']  
weather_collection = db['weather_data']
futures_collection = db['futures_data']


In [5]:
# Define the schema for the weather data
weather_schema = {
    'date': 'date',
    'tavg_fahrenheit': 'double',
    'tmin_fahrenheit': 'double',
    'tmax_fahrenheit': 'double'
}

# Define the schema for the futures data
futures_schema = {
    'Date': 'date',
    'Open': 'double',
    'High': 'double',
    'Low': 'double',
    'Close': 'double',
    'Adj_Close': 'double',
    'Volume': 'int',
    'ATR': 'double'
}

In [6]:
# Specify the directory path where the weather JSON files are located
weather_directory = r'C:\Users\abrid\OneDrive\Desktop\Project 3 Folder\Group-Project-3\data\weather'

# Specify the directory path where the futures JSON files are located
futures_directory = r'C:\Users\abrid\OneDrive\Desktop\Project 3 Folder\Group-Project-3\data\futures'

In [7]:
# Import weather data from multiple files for each city
cities = ['Los Angeles', 'New York City', 'Chicago', 'Detroit', 'Columbus', 'Philadelphia', 'Newark', 'Houston', 'Indianapolis', 'Milwaukee']

for city in cities:
    collection = db['weather_data']  # Use the same collection for all cities
    city_file_path = os.path.join(weather_directory, f'{city}_data.json')
    import_weather_data(city_file_path, weather_schema, collection, city)

In [8]:
# Import futures data from multiple files
futures_files = os.listdir(futures_directory)
for index, file in enumerate(futures_files):
    if file.endswith('.json'):
        label = f'Winter {index + 1}'
        file_path = os.path.join(futures_directory, file)
        import_futures_data(file_path, futures_schema, futures_collection, label)

Processing futures data for Winter 1: [{'Date': '2011-12-01', 'Open': 3.575000047683716, 'High': 3.688999891281128, 'Low': 3.5409998893737793, 'Close': 3.6480000019073486, 'Adj_Close': 3.6480000019073486, 'Volume': 175649, 'ATR': 0.14800000190734863}, {'Date': '2011-12-02', 'Open': 3.6440000534057617, 'High': 3.6630001068115234, 'Low': 3.568000078201294, 'Close': 3.5840001106262207, 'Adj_Close': 3.5840001106262207, 'Volume': 84298, 'ATR': 0.11620001792907714}, {'Date': '2011-12-05', 'Open': 3.569999933242798, 'High': 3.572000026702881, 'Low': 3.444000005722046, 'Close': 3.4609999656677246, 'Adj_Close': 3.4609999656677246, 'Volume': 114253, 'ATR': 0.12178949305885715}, {'Date': '2011-12-06', 'Open': 3.4630000591278076, 'High': 3.506999969482422, 'Low': 3.4049999713897705, 'Close': 3.486999988555908, 'Adj_Close': 3.486999988555908, 'Volume': 122829, 'ATR': 0.11356924130366397}, {'Date': '2011-12-07', 'Open': 3.496999979019165, 'High': 3.503999948501587, 'Low': 3.4089999198913574, 'Close'

In [9]:
# Close the MongoDB connection
client.close()